In [1]:
#heterogeneous quality + heterogeneous rhetorical value full model
# note: in the paper, we run the code below 20 times for each set of
# (literature size, reading budget, reference budget) values.
#The code/result here is for 1 run.

import random
import numpy as np
random.seed(100000)
np.random.seed(100000)
#fix random seeds across six models to make results exactly reproducible
#average across 20 fixed-seed runs and get average effects/error bands, etc.
#this is because we want to ensure that
#observed differences between models
#are due to the mechanisms we proposed and not random variations between comparison experiments

import scipy.stats as stats
from scipy.stats import pearsonr
from scipy.stats import norm


d:\Program\anaconda\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:

#helper function for calculating Gini coefficient
def gini_coefficient(x):
    """Compute Gini coefficient of array of values"""
    diffsum = 0
    for i, xi in enumerate(x[:-1], 1):
        diffsum += np.sum(np.abs(xi - x[i:]))
    return diffsum / (len(x)**2 * np.mean(x))

#helper function for getting indices of the top N values of a list (which to read/cite)
def f(a,N):
    """Get indices of the top N values of a list"""
    return np.argsort(a)[::-1][:N]

In [9]:
# THIS SECTION DETERMINES WHICH PARAMETER WILL VARY (literature size, reference budget, or reading budget).
# To choose which parameter to vary, change the min and max values accordingly. Currently, the code is set to
# vary the reference budget.

tmax= 100 #time steps
#-----------------------------
num = 600 # literature size min
nummax=600 # literature size max
#-----------------------------
reference = 100 # reference budget min
refmax =100  # reference budget max
#-----------------------------
reading = 120 # reading budget min
readingmax=120 # reading budget max
#-----------------------------
noise =0.05 #tested for robustness (see Appendix 1.3)
fit =0.1 #tested for robustness (see Appendix 1.4)
shape =6 #shape for distributions of underlying initial rhetorical value and base quality, tested for robustness (see Appendix 1.1)
#-----------------------------------------------------------------


In [10]:
# OUTCOMES
listcorr=[] #correlation (citation-quality)
listgini=[] #gini
#figure 2, in what way top-quality/mid-quality papers are cited? (substantive or rhetorical)
list1=[] #top papers cited substantively
list2=[] #top papers cited rhetorically
list3=[] #mid papers cited substantively
list4=[] #mid papers cited rhetorically


In [27]:
import tqdm as tq
for num in range(num, nummax+1): # varying literature size
  normative=np.random.beta(1, shape, size=num)   #控制 qi的
  qrank =[]
  qrank = list(np.argsort(normative)[-(num):][::-1]) # 最好的在前面    quality distribution and its rank
  weight=0.001 #the weight of citation count on perceived quality
  weightq =0.3#the perceived quality signal-based gain in rhetoric value
  #tested for robustness (see Appendix 1.5: reinforcing process)

  for reference in tq.tqdm(range(reference, refmax+1)):   # comment if want to fix reference budget
  #for reading in range(reading, readingmax+1):  # uncomment if want to vary reading budget

    top1q=qrank[0:40]# top 40 quality (high quality)
    top2q=qrank[40:int(150)]# top 40-150 quality (mid-to-high quality) /600 in total

    cite_population = [0]*num #citation count over the entire paper population, initial as 0
    rhe_list=[0]*num #rhetoric value, initial as 0
    chunk =[] #citation churn

    for t in range(1, tmax+1): #a reader joins
    #heterogeneous quality: the reader has her own perception of--
    #--threshold/fit/error/underlying rhetorical value
    #initialize them within the loop

      threshold =random.uniform(0,1)
      #normal distribution of threshold for robustness
      #we tested robustness (see Appendix 1.2)
      #mu, sigma = 0.5, 0.2
      #lower, upper = 0, 1
      #X1 = stats.truncnorm(
      #  (lower - mu) / sigma, (upper - mu) / sigma, loc=mu, scale=sigma)
      #threshold1=X1.rvs(1)
      #threshold =threshold1[0]

      #underlying rhetorical value is heterogeneous
      base_rhe=np.random.beta(1, shape, size=num)

      noise_list=np.random.normal(0, noise, num)
      #we tested it for robustness (see Appendix 1.3)

      #fit - heterogeneous
      fit_list=[]
      for i in range(num):
        fit_list.append(random.uniform(-fit, fit))
      #tested for robustness (see Appendix 1.4)

      # quality + fit + perception error, needs to be truncated to [0, 1]
      pq =[]
      for i in range(num):
        #0-1 cut off
        pq.append(normative[i]+ fit_list[i] + noise_list[i])
      for i in range(num):
        if (pq[i] >1): #maximum 1, quality +fit +error =1
          pq[i] =1
        if (pq[i] <0):
          pq[i] =0 #non-negative

      # perceived quality signal (quality + fit + perception error + citation premium)
      signal =[]
      for i in range(num):
        signal.append(pq[i] + weight * cite_population[i] )

      # the conditions below ensure that perceived quality signal is within [0, 2]
      # but given the constituent parts it should never be outside of that anyway
      for i in range(num):
        if (signal[i] >2): #maximum 2, quality +fit +error =1, weight * citation (citation premium) =1
          signal[i] =2
        if (signal[i] <0):
          signal[i] =0 #non-negative

      #read papers with the highest perceived quality
      reading_index = list(np.argsort(signal)[-(reading):][::-1])

      # CALCULATE OVERALL RHETORICAL VALUE
      # rhetorical value = base rhetorical value + beta*perceived quality
      # but perceived quality should be adjusted if paper is read (i.e., perception error removed)
      # If read...
      #PAPER IS READ:
      # remove perception noise from the rhetorical value of *read* papers (since their quality has been observed)
      for i in range(len(reading_index)):
        rhe_list[reading_index[i]] =base_rhe[reading_index[i]] + weightq* (pq[reading_index[i]] -noise_list[reading_index[i]] ) + weightq* weight*cite_population[reading_index[i]]

      # If unread...
      unread=[]
      unread = [i for i in list(range(0,num)) if i not in reading_index]
      for i in range(len(unread)):
        rhe_list[unread[i]] =base_rhe[unread[i]] + weightq* pq[unread[i]]+ weightq* weight*cite_population[unread[i]]

      # truncate rhetorical values to [0, 1+2*beta]
      # but given the constituent parts it should never be outside of that anyway
      for i in range(num):
        if (rhe_list[i] <0):
          rhe_list[i] =0
        if(rhe_list[i] > 1+ weightq+ weightq):
          rhe_list[i] =1+ weightq+ weightq

      # perceived quality: after reading, the error disappears, so perceived quality = quality + fit
      norm_list =[]
      for i in range(len(reading_index)):
        norm_list.append(normative[reading_index[i]] + fit_list[reading_index[i]])

      # truncate perceived quality to [0, 1]
      # the idea behind all truncations is that
      # fit or perception error can make a paper perfect (perceived quality close to 1) in people's eyes
      # but they cannot make an already perfect one exceed the range
      for i in range(len(reading_index)):
        if (norm_list[i] <0):
          norm_list[i] =0
        if (norm_list[i] >1):
          norm_list[i] =1

      #which paper is above the substantive citing threshold?
      over_threshold=[]
      for i in range(len(norm_list)):
        if (norm_list[i]> threshold):
          over_threshold.append(reading_index[i])

      normative_cite=[]# substantive citation list
      rhetorical_cite=[]# rhetorical citation list
      overlap=[]#overlap between substantive and rhetorical citing
      overall_cite =[]

      #if there are enough good papers for substantive citing
      #then all cites are substantive (cite the best ones within the citing budget)
      #update citation counts -- and the impact that citations induce
      if (len(over_threshold) >= reference):
        cite = list(f(norm_list, reference)) #获得前reference的最大的数
        for i in range(len(cite)):
          cite_population[reading_index[cite[i]]]= cite_population[reading_index[cite[i]]]+1
          normative_cite.append(reading_index[cite[i]])
        rhetorical_cite=[]
        overlap =[]
        overall_cite = normative_cite + rhetorical_cite + overlap

      #if there are insufficiently many good papers for substantive citing
      #first, cite all of these good ones substantively as "normative_cite"
      else:
        normative_cite = over_threshold.copy()
        #how many slots are left -- we fill up all of the rest slots according to the overall rhetorical values
        rhetoric_no = reference - len(over_threshold)
        new_rhe = rhe_list.copy()
        rhe2=[]
        rhe2 = sorted(new_rhe, reverse = True)
        itr=0
        itr2=0
        while (itr < rhetoric_no):
          # a small proportion of papers first is cited substantively
          # then if people find them rhetorically useful as well
          # move them into the set of "overlap" --
          # cited both substantively and rhetorically (we allow this which is the case in the real world)

          if ((new_rhe.index(rhe2[itr2]) in normative_cite) == True):
            normative_cite.remove(new_rhe.index(rhe2[itr2]))
            numitr = new_rhe.index(rhe2[itr2])
            overlap.append(numitr)
            itr2 =itr2+1
          else:
            rhetorical_cite.append(new_rhe.index(rhe2[itr2]))
            itr =itr+1
            itr2 =itr2+1

        #update citation count
        overall_cite = normative_cite + rhetorical_cite + overlap
        for i in range(len(overall_cite)):
          cite_population[overall_cite[i]] = cite_population[overall_cite[i]] +1
      #churn
      #this round, which papers get cited?
      chunk.append(overall_cite)

# SUB-ANALYSIS
# track outcomes for 2 types of papers {high quality, mid quality}
# and 2 types of citations {substantive, rhetorical}, see Figure 2
      #top1=0
      #for i in range(40):
      # if ((top1q[i] in (normative_cite + overlap)) == True):
      #    top1 = top1+1
      #list1.append(top1/reference)
      #top2=0
      #for i in range(40):
      #  if ((top1q[i] in (rhetorical_cite + overlap)) == True):
      #    top2 = top2+1
      #list2.append(top2/reference)
      #top3=0
      #for i in range(110):
      #  if ((top2q[i] in (normative_cite + overlap)) == True):
      #    top3 = top3+1
      #list3.append(top3/reference)
      #top4=0
      #for i in range(110):
      #   if ((top2q[i] in (rhetorical_cite + overlap)) == True):
      #    top4 = top4+1
      #list4.append(top4/reference)

    #print(*cite_population, sep='\n') #every round the citation count distribution

    #listgini.append(gini_coefficient(np.array(cite_population))) #gini coefficient

    corr1, _ = stats.pearsonr(normative, cite_population)
    listcorr.append(corr1) #citation-quality correlation, example output

    # churn, which is the newly cited papers compared to the last round
    dnew=[]
    for i in range(len(chunk) -1):
      a=0
      for j in range(len(chunk[i+1])):
       if ((chunk[i+1][j] in chunk[i]) == False):
         a=a+1
      dnew.append(a)
    print(np.mean(dnew))

#print(list1)
#print(list2)
#print(list3)
#print(list4)
print('example output: Correlation (citation-quality)')
print(*listcorr, sep='\n')
print('Note: the full model\'s correlation is higher than either null models, but the null models are in different notebooks')
#print(listgini)

100%|██████████| 1/1 [00:00<00:00,  2.49it/s]

69.70707070707071
example output: Correlation (citation-quality)
0.9134441681116476
0.911429806469696
Note: the full model's correlation is higher than either null models, but the null models are in different notebooks


In [22]:
len(list(f(norm_list, reference)))

100

In [23]:
reference

100

In [24]:
list(f([1,2,3], 1))

[2]